<a href="https://colab.research.google.com/github/Mark-Campanella/Newscraper-PySelenium/blob/main/Newscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller
pip install selenium
pip install textsum

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Get:13 http://archive.ubuntu.com/ubuntu j

In [2]:
import csv
import json
import os
import pandas as pd
import re
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from textsum.summarize import Summarizer
import time
import sys
from flask import Flask, render_template, request, send_file

In [22]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
path_csv_data = '/content/drive/MyDrive/Colab Notebooks/Newscraper/CSV/data.csv'
path_csv_lookup = '/content/drive/MyDrive/Colab Notebooks/Newscraper/CSV/lookup.csv'
path_json_data = '/content/drive/MyDrive/Colab Notebooks/Newscraper/JSON/data.json'
template_folder = '/content/drive/MyDrive/Colab Notebooks/Newscraper/templates'

In [21]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/125/chromedriver'

In [23]:
def flush_data():
    '''
    Deletes the data from all files
    Returns whether the data was flushed or not
    '''
    try:
        with open(path_json_data, 'w', encoding='utf-8') as json_file:
            json_file.write('')

        with open(path_csv_data, 'w', encoding='utf-8') as csv_file:
            csv_file.write('')
        return "Data flushed successfully!"
    except Exception as e:
        return f"Error flushing data: {e}"

In [24]:
def save_to_pre_file(pre_file, item_name, items):
    '''
    Save desired items to the pre_file dictionary

    Parameters:
        - pre_file: dictionary to store data temporarily
        - item_name: type of the object added, e.g.: URL, Title, Text, Scoop, etc.
        - items: list of objects to be added
    '''
    if item_name not in pre_file:
        pre_file[item_name] = []
    pre_file[item_name].extend(items)


In [25]:
def rework_data(pre_file):
    '''
    Rework the pre_file data so it is organized into a list of dictionaries
    '''
    data = []
    titles = pre_file.get("Titles", [])
    texts = pre_file.get("Text", [])

    for title, text in zip(titles, texts):
        data.append({"Titles": clean_text(title), "Text": clean_text(text)})

    return data


In [26]:
def clean_text(text):
    '''
    Cleans the text (special chars etc.)!

    Parameters:
        - str text
    '''
    return re.sub(r'[^\x00-\x7F]+', '', text)


In [27]:
def add_year_month(data):
    '''
    It is considered that every news that we are scraping are new, so we get current month and year
    '''
    today = time.asctime(time.localtime())
    year = today[-4:]
    month = today[4:7]

    for item in data:
        item["Year"] = year
        item["Month"] = month

    print("Year and Month added.")
    return data

def sum_text(text):
    '''
    This sum up the texts!

    Parameters:

        - str Text
    '''
    sum = Summarizer()
    scoop = sum.summarize_string(text)
    return scoop

def add_scoop(data):
    for item in data:
        if "Text" in item and "Scoop" not in item:
            original_text = item["Text"]
            summarized_text = sum_text(original_text)
            item["Scoop"] = summarized_text
    return data

def add_urls(data, urls):
    '''
    Adds the urls to the json objects, after everything is already there in the object

    Parameters:
        - list data: list of data dictionaries
        - list urls: list of URLs
    '''
    for i, item in enumerate(data):
        if i < len(urls):
            item["Link"] = urls[i]

    print("URLs added.")
    return data

def try_find_country_brand(data):
    '''
    Looks if the country, region, or continent is mentioned (checked if it is in a lookup table)
    Looks if a company is mentioned
    If they are mentioned (anyone) it is added to the json
    '''
    lookup_csv = path_csv_lookup

    try:
        df = pd.read_csv(lookup_csv)

        for item in data:
            title = item.get("Titles", "")
            text = item.get("Text", "")
            try:
                country_found = False
                for country in df["Country"]:
                    if country in text:
                        item["Country"] = country
                        country_found = True
                        break
                if not country_found: item["Country"] = ""
            except: continue
            try:
                for brand in df["Brand"]:
                    if brand in title or brand in text:
                        item["Brand"] = brand
                        break
                if not item.get("Brand"): item["Brand"] = ""
            except: continue

        print("Brand and/or country added.")
    except Exception as e:
        print("An error occurred:", e)
    return data


In [28]:
def wait_until_page_loads(driver, timeout=30):
    """
    Awaits page to load dynamically

    Parameters:
        - driver: Selenium's driver object
        - int timeout: in seconds, if none is given =  30
    """
    start_time = time.time()
    while True:
        end_time = time.time()
        if (end_time - start_time) > timeout or driver.execute_script("return document.readyState") == "complete":
            break
        time.sleep(3)

In [29]:
def json_to_csv():
    """
    Convert JSON data to CSV format with a specific separator.
    """
    json_file = path_json_data
    csv_file = path_csv_data

    if not os.path.exists(json_file):
        print("JSON file does not exist.")
        return

    with open(json_file, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        header = ["Year", "Month", "Corporate", "Brand", "Att 1", "Att 2", "Att 3", "Product", "Region", "Country",
                  "Link", "LINK 2", "Titles", "Corp", "Flag", "Scoop", "Article photo", "x Photo 2",
                  "x monthly one PAGE LINK", "Monthly newsletter", "Text", "Type of News"]
        writer.writerow(header)

        for row in json_data:
            writer.writerow([row.get(col, '').replace(';', ',') for col in header])

    print(f"CSV file '{csv_file}' has been created.")

In [30]:
def go_into_website(urls):
    '''
    Uses Selenium to get into each webpage in the list of URLs provided by the user
    Gather the title (h1) and the texts (p) from each webpage and close the navigator
    It calls rework_data() to clean up the pre_file and save the final data

    Parameters:
    list urls: list of URLs of the news websites
    '''
    pre_file = {}
    for url in urls:
        if not url.startswith("http://") and not url.startswith("https://"):
            raise ValueError(f"Invalid URL: {url}")
        driver = webdriver.Chrome(options=chrome_options)
        titles = []
        text = []
        try:
            driver.get(url)
            wait_until_page_loads(driver)

            elems = driver.find_elements(By.TAG_NAME, 'h1')
            titles = [elem.text for elem in elems]
            save_to_pre_file(pre_file, "Titles", titles)

            elems = driver.find_elements(By.TAG_NAME, 'p')
            text = [elem.text for elem in elems]
            save_to_pre_file(pre_file, "Text", text)

            print(f"\n-----------------------------------\nWebsite info for {url} got copied!\n-------------------------------------------")
        except Exception as e:
            print(f"ERROR!: {e}")
        finally:
            driver.quit()

    try:
        data = rework_data(pre_file)
        data = add_scoop(data)
        data = add_urls(data, urls)
        data = try_find_country_brand(data)
        data = add_year_month(data)

        with open(path_json_data, 'w', encoding='utf-8') as output_file:
            json.dump(data, output_file, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"An error occurred: {e}")


In [31]:
from google.colab import output
output.serve_kernel_port_as_window(8084)

<IPython.core.display.Javascript object>

In [32]:
#----------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------#
#-------------------------------------------Backend for the user input-------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------#

newscrapper = Flask(__name__, template_folder=template_folder)

@newscrapper.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if request.form.get('flush_data_button'):
            result = flush_data() #Delete data from csv and jsons
            return render_template('index.html', user_input=result)

        user_input = request.form['user_input']
        # Converts to a list of urls
        urls = user_input.split(',')  # URLs should be separated by commas!!
        try:
            go_into_website(urls)
            json_to_csv()  # Convert JSON to CSV
            return send_file(path_csv_data, as_attachment=True)
        except:
            return render_template('index.html', user_input="Website was not found!")
    return render_template('index.html', user_input="")

if __name__ == '__main__':
    newscrapper.run(host='0.0.0.0', port=8084)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8084
 * Running on http://172.28.0.12:8084
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/May/2024 16:47:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/May/2024 16:47:30] "GET /favicon.ico HTTP/1.1" 404 -



-----------------------------------
Website info for https://www.channelnews.com.au/haier-appliance-profits-soar-as-revenues-climb-new-oz-developed-energy-washing-machine-contributing/?utm_campaign=haier-appliance-profits-soar-as-revenues-climb-new-oz-developed-energy-washing-machine-contributing&utm_medium=rss&utm_source=rss got copied!
-------------------------------------------

-----------------------------------
Website info for https://economictimes.indiatimes.com/industry/cons-products/durables/german-home-appliances-maker-bsh-hausgerte-gmbh-considers-india-its-2nd-largest-emerging-market/articleshow/109241796.cms?from=mdr got copied!
-------------------------------------------


Generating Summaries:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1006: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Generating Summaries:   0%|          | 0/1 [00:00<?, ?it/s]

URLs added.


INFO:werkzeug:127.0.0.1 - - [24/May/2024 16:48:35] "POST / HTTP/1.1" 200 -


Brand and/or country added.
Year and Month added.
CSV file '/content/drive/MyDrive/Colab Notebooks/Newscraper/CSV/data.csv' has been created.


INFO:werkzeug:127.0.0.1 - - [24/May/2024 16:56:04] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/May/2024 16:56:05] "GET /favicon.ico HTTP/1.1" 404 -
